# Drugi domaći zadatak iz Digitalne obrade signala
## Student: Vojin Vukčević 2021/0516

# Deo 1: Filtriranje zvučnog signala

In [ ]:
USE_WIDGETS = True

import numpy as np
if USE_WIDGETS:
    #from google.colab import output
    #output.enable_custom_widget_manager()
    %matplotlib widget
else:
    %matplotlib inline
import matplotlib as mpl
mpl.rc('text', usetex = False)
mpl.rc('font', family = 'serif', size = 16)
import matplotlib.pyplot as plt
import pickle
import scipy.signal as signal
import scipy.io as sio
import scipy.fft as fft


In [ ]:
from scipy.io import wavfile
import IPython

fs1, allplusA = wavfile.read('dz2_signali/allplusA.wav')
#IPython.display.display(IPython.display.Audio(allplusA, rate = fs1))
fs2, noteA = wavfile.read('dz2_signali/noteA.wav')
#IPython.display.display(IPython.display.Audio(noteA, rate = fs2))
fig, ax = plt.subplots(2,1, figsize=[11,7])
ax[0].plot(np.arange(len(allplusA))/fs1, allplusA)
ax[1].plot(np.arange(len(noteA))/fs2, noteA)
ax[0].set_title('Signal "allplusA.wav"')
ax[1].set_title('Signal "noteA.wav"')
ax[0].set_xlabel('$t[s]$')
ax[1].set_xlabel('$t[s]$')
ax[0].grid()
ax[1].grid()
plt.tight_layout()
plt.show()

In [ ]:
import math

def bandstop_filter_Cheb2(fs, fa, fp, Aa, Ap):
    Ts = 1/fs
    #gabariti naseg digitalnog filtra
    Wa1 = 2*np.pi*fa[0]/fs
    Wa2 = 2*np.pi*fa[1]/fs
    Wp1 = 2*np.pi*fp[0]/fs
    Wp2 = 2*np.pi*fp[1]/fs
    
    #gabariti naseg analognog filtra
    wa1 = (2/Ts) * np.tan(Wa1/2)
    wa2 = (2/Ts) * np.tan(Wa2/2)
    wp1 = (2/Ts) * np.tan(Wp1/2)
    wp2 = (2/Ts) * np.tan(Wp2/2)
    
    #normalizovana specifikacija
    waN = 1

    w0 = np.sqrt(wa1*wa2)
    B = wa2 - wa1
    
    wpN = abs(((wa2 - wa1)*wp1)/(w0**2 - wp1**2))

    D = (10**(0.1*Aa)-1)/(10**(0.1*Ap)-1)
    k = waN/wpN
    N = int(np.ceil( np.arccosh(np.sqrt(D))/np.arccosh(k)))
    
    z, p, k = signal.cheb2ap(N, Aa)
    
    an = np.array(np.poly(p))
    bn = k*np.array(np.poly(z))
    if an.shape == ():
        an = np.array([an])
    if bn.shape == ():
        bn = np.array([bn])
    
    b, a = signal.lp2bs(bn, an, w0, B);
    
    [bd, ad] = signal.bilinear(b, a, fs)
    
    return bd, ad

In [ ]:
f1,t1,S1 = signal.spectrogram(allplusA, fs1, window = 'hann', nperseg=512, noverlap = 256, nfft= 2048)
S1_dB = 20 * np.log10(np.abs(S1) + 1e-12) #amplituda u dB
f2,t2,S2 = signal.spectrogram(noteA, fs2, window = 'hann', nperseg=512, noverlap = 256, nfft= 2048)
S2_dB = 20 * np.log10(np.abs(S2) + 1e-12) #amplituda u dB
fig, ax = plt.subplots(1, 2, figsize=(12, 6))
ax[0].pcolormesh(t1, f1, S1_dB, shading='gouraud', cmap = 'inferno')
ax[1].pcolormesh(t2, f2, S2_dB, shading='gouraud', cmap = 'inferno')
ax[0].set_xlabel("Vreme $[s]$")
ax[0].set_ylabel("Frekvencija $[Hz]$")
ax[0].set_title('Spektrogram "allplusA.wav" signala')
ax[1].set_xlabel("Vreme $[s]$")
ax[1].set_ylabel("Frekvencija $[Hz]$")
ax[1].set_title('Spektrogram "noteA.wav" signala')
ax[0].set_ylim(0,10000)
ax[1].set_ylim(0,10000)
plt.tight_layout()
plt.show()

In [ ]:
fa1 = [400, 480]   
fp1 = [300, 580]

Ap = 1            
Aa = 45

bd, ad = bandstop_filter_Cheb2(fs1, fa1, fp1, Aa, Ap)
filtered_allplusA = signal.lfilter(bd, ad, allplusA)

fa2 = [840, 920]   
fp2 = [740, 1020]

bd2, ad2 = bandstop_filter_Cheb2(fs1, fa2, fp2, Aa, Ap)
filtered_allplusA2 = signal.lfilter(bd2, ad2, filtered_allplusA)

fa3 = [180, 260]   
fp3 = [80, 360]

bd3, ad3 = bandstop_filter_Cheb2(fs1, fa3, fp3, Aa, Ap)
filtered_allplusA3 = signal.lfilter(bd3, ad3, filtered_allplusA2)

fa4 = [1730, 1790]   
fp4 = [1630, 1890]

bd4, ad4 = bandstop_filter_Cheb2(fs1, fa4, fp4, Aa, Ap)
filtered_allplusA4 = signal.lfilter(bd4, ad4, filtered_allplusA3)

fa5 = [3450, 3600]   
fp5 = [3350, 3700]

bd5, ad5 = bandstop_filter_Cheb2(fs1, fa5, fp5, Aa, Ap)
filtered_allplusA5 = signal.lfilter(bd5, ad5, filtered_allplusA4)

fa6 = [6940, 7140]   
fp6 = [6840, 7270]

bd6, ad6 = bandstop_filter_Cheb2(fs1, fa6, fp6, Aa, Ap)
filtered_allplusA6 = signal.lfilter(bd6, ad6, filtered_allplusA5)


IPython.display.display(IPython.display.Audio(filtered_allplusA6, rate = fs1))

f1,t1,S1 = signal.spectrogram(filtered_allplusA6, fs1, window = 'hann', nperseg=512, noverlap = 256, nfft= 2048)
S1_dB = 20 * np.log10(np.abs(S1) + 1e-12) #amplituda u dB
plt.figure(figsize=(10, 6))
plt.pcolormesh(t1, f1, S1_dB, shading='gouraud', cmap = 'inferno')
plt.xlabel("Vreme $[s]$")
plt.ylabel("Frekvencija $[Hz]$")
plt.title("Spektrogram chirp signala")
plt.colorbar(label="Amplituda $[dB]$")
plt.ylim(0,10000)
plt.tight_layout()
plt.show()

# Deo 2: Ekvalizacija zvučnog signala

In [ ]:
def bandpass_filter(fs, fa, fp, Aa, Ap):
    Ts = 1/fs
    #gabariti naseg digitalnog filtra
    Wa1 = fa[0]/fs*2*np.pi
    Wp1 = fp[0]/fs*2*np.pi
    Wp2 = fp[1]/fs*2*np.pi
    Wa2 = fa[1]/fs*2*np.pi

    # Dizajn filtra
    #normalizacija parametra zbog iirdesign potreba
    wpVector = np.array([Wp1, Wp2])/np.pi;
    waVector = np.array([Wa1, Wa2])/np.pi;
    b, a = signal.iirdesign(wpVector, waVector, Ap, Aa, analog=False, ftype='ellip', output='ba', fs = None)
    
    
    return b, a

In [ ]:
def highpass_filter(fs, fa, fp, Aa, Ap):
    
    Ts = 1/fs
    #gabariti naseg digitalnog filtra
    Wa = fa/fs*2*np.pi
    Wp = fp/fs*2*np.pi
    
    # Dizajn filtra
    #normalizacija parametra zbog iirdesign potreba
    Wa_norm = Wa/np.pi
    Wp_norm = Wp/np.pi
    
    b, a = signal.iirdesign(Wp_norm, Wa_norm, Ap, Aa, analog=False, ftype='ellip', output='ba', fs = None)
    
    return b, a

In [ ]:
fs = 44100  
Aa = 60
Ap = 0.05

fp_bp = [8000, 16000]
fa_bp = [7800, 16200]

bd_bp, ad_bp = bandpass_filter(fs, fa_bp, fp_bp, Aa, Ap)

fa_hp = 15800  
fp_hp = 16000

bd_hp, ad_hp = highpass_filter(fs, fa_hp, fp_hp, Aa, Ap)

In [ ]:
import math
import matplotlib.patches as patches

# Frekvencijska karakteristika
w_bp, H_bp = signal.freqs(bd_bp, ad_bp, worN = 10000)
f_bp = w_bp/2/np.pi
fd_bp, Hd_bp = signal.freqz(bd_bp, ad_bp, worN=10000, fs=fs)

# Amplitudska karakteristika
Ha_bp = abs(H_bp)
Hda_bp = abs(Hd_bp)
faza = np.angle(Hd_bp)
faza = np.unwrap(np.angle(Hd_bp))

rXlim = 25000

fig, ax = plt.subplots(2,1, figsize=(12,6))

#crtanje gabarita filtra
# First stop band lim
ax[0].add_patch(patches.Polygon([[0, -Aa ], [fa_bp[0], -Aa], [fa_bp[0], -Aa+5], [0, -Aa+5]], closed=True,
                                  fill=False, hatch='////', color = 'red'))
# Pass band lim
ax[0].add_patch(patches.Polygon([[fp_bp[0], -Ap-5 ], [fp_bp[1], -Ap-5], [fp_bp[1], -Ap], [fp_bp[0], -Ap]], closed=True,
                                  fill=False, hatch='////', color = 'red'))
# Second stop band lim
ax[0].add_patch(patches.Polygon([[fa_bp[1], -Aa ], [rXlim, -Aa], [rXlim, -Aa+5], [fa_bp[1], -Aa+5]], closed=True,
                                  fill=False, hatch='////', color = 'red'))
#plot Bandpass filtra
ax[0].set_title('Amplitudska karakteristika Bandpass filtra')
ax[1].set_title('Fazna karakteristika Bandpass filtra')
ax[0].set_xlabel('$f$ [Hz]')
ax[0].set_ylabel('Amplituda [dB]')
ax[1].set_xlabel('$f$ [Hz]')
ax[1].set_ylabel('Faza [rad]')
ax[0].plot(fd_bp, 20*np.log10(Hda_bp))
ax[1].plot(fd_bp, faza)
ax[0].set_ylim(-65,5)
ax[0].grid()
ax[1].grid()
plt.tight_layout()
plt.show()

# Frekvencijska karakteristika
w_hp, H_hp = signal.freqs(bd_hp, ad_hp, worN = 10000)
f_hp = w_hp/2/np.pi
fd_hp, Hd_hp = signal.freqz(bd_hp, ad_hp, worN=10000, fs=fs)
faza_hp = np.angle(Hd_hp)
faza_hp = np.unwrap(np.angle(Hd_hp))
# Amplitudska karakteristika
Ha_hp = abs(H_hp)
Hda_hp = abs(Hd_hp)
fig, ax = plt.subplots(2,1, figsize=(12,6))
#crtanje gabarita filtra
ax[0].add_patch(patches.Polygon([[0, -Aa ], [fa_hp, -Aa], [fa_hp, -Aa+5], [0, -Aa+5]], closed=True,
                                          fill=False, hatch='////', color = 'red'))
# Pass band lim
ax[0].add_patch(patches.Polygon([[fp_hp, -Ap-5 ], [rXlim, -Ap-5], [rXlim, -Ap], [fp_hp, -Ap]], closed=True,
                                          fill=False, hatch='////', color = 'red'))
#plot Highpass filtra
ax[0].set_title('Amplitudska karakteristika Highpass filtra')
ax[1].set_title('Fazna karakteristika Highpass filtra')
ax[0].set_xlabel('$f$ [Hz]')
ax[0].set_ylabel('Amplituda [dB]')
ax[1].set_xlabel('$f$ [Hz]')
ax[1].set_ylabel('Faza [rad]')
ax[0].plot(fd_hp, 20*np.log10(Hda_hp))
ax[1].plot(fd_hp, faza_hp)
ax[0].set_ylim(-65,5)
ax[0].grid()
ax[1].grid()
plt.tight_layout()
plt.show

In [ ]:
def IIR_equalizer(x, fs, style):
    #kreiranje ulaznih signala
    N = len(x)
    #prvo radimo decimaciju 
    x1 = x
    x2 = signal.resample_poly(x, 1, 2) 
    x4 = signal.resample_poly(x, 1, 4)
    x8 = signal.resample_poly(x, 1, 8)
    x16 = signal.resample_poly(x, 1, 16)
    x32 = signal.resample_poly(x, 1, 32)
    x64 = signal.resample_poly(x, 1, 64)
    x128 = signal.resample_poly(x, 1, 128)
    x256 = signal.resample_poly(x, 1, 256)
    
    #filteri:
    #BAND-PASS:
    b1, a1 = bandpass_filter(fs, [7800, 16200], [8000, 16000], Aa, Ap) 
    #HIGH-PASS: 
    b2, a2 = highpass_filter(fs, 15800, 16000, Aa, Ap)     
    
    #kreiramo pojacanje za zadati stil
    if(style == 'ROCK'):
        POJACANJE = [5, 3.75, 3, 1.5, -0.5, -1.5, 1, 2.5, 3.75, 4.5]
    elif(style == 'POP'):
        POJACANJE = [1.5, -1, 0, 1.5, 4, 4, 2, 0, -1, -1.5]
    elif(style == 'DANCE'):
        POJACANJE = [4, 7, 5, 0, 2, 4, 5, 4.5, 3.5, 0]
    elif (style == 'CUSTOM'):
        POJACANJE = [7, 8, 4, -2, -3, 1, 3, 3, 4, 2]
    #racunamo pojacanje u decibelima
    POJACANJE = [10**(db/20) for db in POJACANJE]
    
    #primenjujemo napravljene filtre na nase ulaze
    x1_eq = lfilter( b1, a1, x256*POJACANJE[0])
    x2_eq = lfilter( b1, a1, x128*POJACANJE[1])
    x3_eq = lfilter( b1, a1, x64*POJACANJE[2])
    x4_eq = lfilter( b1, a1, x32*POJACANJE[3])
    x5_eq = lfilter( b1, a1, x16*POJACANJE[4])
    x6_eq = lfilter( b1, a1, x8*POJACANJE[5])
    x7_eq = lfilter( b1, a1, x4*POJACANJE[6])
    x8_eq = lfilter( b1, a1, x2*POJACANJE[7])
    x9_eq = lfilter( b1, a1, x1*POJACANJE[8])
    x10_eq = lfilter( b2, a2, x*POJACANJE[9])
    
    #vrsimo interpolaciju i zbir je nas trazeni obradjeni signal
    x_total_eq = signal.resample_poly(x1_eq, 256, 1)[:N]  + signal.resample_poly(x2_eq, 128, 1)[:N]  + signal.resample_poly(x3_eq, 64, 1)[:N] + signal.resample_poly(x4_eq, 32, 1)[:N] + signal.resample_poly(x5_eq, 16, 1)[:N] + signal.resample_poly(x6_eq, 8, 1)[:N] + signal.resample_poly(x7_eq, 4, 1)[:N] + signal.resample_poly(x8_eq, 2, 1)[:N] + signal.resample_poly(x9_eq, 1, 1)[:N] + x10_eq[:N]
    return x_total_eq


In [ ]:
from scipy.signal import lfilter
#kreiramo imuplsni signal
impuls = np.zeros(2**16)
impuls[0] = 1
h = IIR_equalizer(impuls, fs, 'ROCK')
fig, ax = plt.subplots(figsize = (12,6))
ax.set_title('Impulsni odziv ekvalizatora')
ax.set_ylabel('h[n]')
ax.set_xlabel('n')
ax.plot(np.arange(len(h))[:2048],h[:2048]) 
plt.tight_layout()
plt.grid()
plt.show()

H = fft.fft(h)
H_db = 20 * np.log10(np.abs(H) + 1e-6)
f_full = np.arange(0,fs,fs/(len(H)))
fig, ax = plt.subplots(figsize = (12,6))
ax.set_title('Frekvencijska karakteristika ekvalizatora')
ax.set_ylabel('Amplituda [dB]')
ax.set_xlabel('[Hz]')
ax.set_ylim(-7,7)
ax.plot(f_full[:len(h)//2], H_db[:len(h)//2])
plt.tight_layout()
plt.grid()
plt.show

In [ ]:
fs, rock = wavfile.read('dz2_signali/rock.wav')

rock_rock = IIR_equalizer(rock, fs, 'ROCK')
print('Ovo je ROCK ekvalizacija:')
IPython.display.display(IPython.display.Audio(rock_rock, rate = fs))
rock_pop = IIR_equalizer(rock, fs, 'POP')
print('Ovo je POP ekvalizacija:')
IPython.display.display(IPython.display.Audio(rock_pop, rate = fs))
rock_dance = IIR_equalizer(rock, fs, 'DANCE')
print('Ovo je DANCE ekvalizacija:')
IPython.display.display(IPython.display.Audio(rock_dance, rate = fs))
rock_custom = IIR_equalizer(rock, fs, 'CUSTOM')
print('Ovo je CUSTOM ekvalizacija:')
IPython.display.display(IPython.display.Audio(rock_custom, rate = fs))

# Deo 3: Promena učestanosti odabiranja:

In [ ]:
def lowpass_filter_kaiser(Wc, Bt, Aa, Ap):
    #granicni parametri naseg filtra 
    Wp = Wc - Bt/2
    Wa = Wc + Bt/2
    #delta
    delta_p = (10**(0.05*Ap)-1)/(10**(0.05*Ap)+1)
    delta_a = 10**(-0.05*Aa)
    delta = min([delta_p, delta_a])
    
    # Alpha_a
    if delta < delta_a:
        AaSpec = -20*np.log10(delta)
    else:
        AaSpec = Aa
    
    # Beta
    beta = 0;
    if AaSpec < 21:
        beta = 0
    elif AaSpec <= 50:
        beta = 0.5842*(AaSpec-21)**0.4 + 0.07886*(AaSpec-21)
    else:
        beta = 0.1102*(AaSpec-8.7)
    
    # Filter order
    if AaSpec <= 21:
        D = 0.9222
    else:
        D = (AaSpec-7.95)/14.36
    M = int(np.ceil(2*np.pi*D/Bt + 1))
    N = M - 1
    #while koji pravi filtar, zatim ispituje da li su svi gabariti ispunjeni, ako nisu povecava red filtra za 1, i ponovo vrsi proveru
    filterFitsInSpecs = False
    first = True
    while not filterFitsInSpecs:
        w = signal.kaiser(N+1, beta)
        na = np.arange(-N/2, N/2+1, 1)
        n = np.arange(N+1)
        hID = np.zeros(N+1)
        
        if N % 2 == 0:
            na = np.delete(na, N//2) # remove index 0
            hID[n != N//2] = np.sin(Wc*na)/(na*np.pi)
            hID[N//2] = Wc/np.pi # lim in na->0
        else:
            hID = np.sin(Wc*na)/(na*np.pi)
    
        h = hID * w
        numPoints = 16000
        W, H = signal.freqz(h, 1, worN = numPoints)
        Hdb = 20*np.log10(abs(H))
        if first:
            first = False
        
        DeltaW = np.pi / numPoints
        kPB = int(np.floor(Wp / DeltaW))  
        kSB = int(np.ceil(Wa / DeltaW))  
        
        if np.all(Hdb[:kPB] > -Ap) and np.all(Hdb[:kPB] < Ap) and np.all(Hdb[kSB:] < -Aa):
            filterFitsInSpecs = True
        else:
            N += 1
        
    return h

In [ ]:
def dos_resample(x, up_down, r):
    #ako je r = 1 nepotrebno je raditi resample i dodavati kasnjenje
    if r==1:
        return x
    #kreiranje filtra za zadate vrednosti iz zadataka
    h_filtra = lowpass_filter_kaiser(np.pi/r, 0.4*np.pi/r, 70, 0.05)
    #koeficijen kasnjenja
    k = len(h_filtra)//2
    if (up_down):
        #ako je decimacija 
        #primenimofiltar, uklonimo kasnjenje i onda uzimamo svaki r-ti odbirak kako bi izvrsisli decimaciju
        x_filtrirano = lfilter(h_filtra,1,x)
        x_bez_kasnjenja = x_filtrirano[k:-k]
        x_finalno = x_bez_kasnjenja[::r]
    else:
        #ako je interpolacija
        #prvo prosirujemo nulama, zatim primenjjemo filtar i na kraju uklonimo kasnjenje
        x_prosireno = np.zeros(len(x)*r)
        x_prosireno[::r] = x
        x_filtrirano = lfilter(h_filtra, 1, x_prosireno)
        # gubimo snagu zbog dodavanja 0, korekciju vrsimo mnozenjem sa r svaki clan
        x_filtrirano = np.array(x_filtrirano) * r
        x_finalno = x_filtrirano[k:-k]
    return x_finalno

In [ ]:
def IIR_equalizer2(x, fs, style):
    #kreiranje ulaznih signala
    N = len(x)
    #prvo radimo decimaciju 
    x1 = x
    x2 = dos_resample(x, True, 2) 
    x4 = dos_resample(x, True, 4)
    x8 = dos_resample(x, True, 8)
    x16 = dos_resample(x, True, 16)
    x32 = dos_resample(x, True, 32)
    x64 = dos_resample(x, True, 64)
    x128 = dos_resample(x, True, 128)
    x256 = dos_resample(x, True, 256)
    
    #filteri:
    #BAND-PASS:
    b1, a1 = bandpass_filter(fs, [7800, 16200], [8000, 16000], Aa, Ap) 
    #HIGH-PASS: 
    b2, a2 = highpass_filter(fs, 15800, 16000, Aa, Ap)     
    
    #kreiramo pojacanje za zadati stil
    if(style == 'ROCK'):
        POJACANJE = [5, 3.75, 3, 1.5, -0.5, -1.5, 1, 2.5, 3.75, 4.5]
    elif(style == 'POP'):
        POJACANJE = [1.5, -1, 0, 1.5, 4, 4, 2, 0, -1, -1.5]
    elif(style == 'DANCE'):
        POJACANJE = [4, 7, 5, 0, 2, 4, 5, 4.5, 3.5, 0]
    elif (style == 'CUSTOM'):
        POJACANJE = [1, 2, 3, 4, 5, 4, 3, 2, 1, 0]
        
    #racunamo pojacanje u decibelima
    POJACANJE = [10**(db/20) for db in POJACANJE]
    
    #primenjujemo napravljene filtre na nase ulaze
    x1_eq = lfilter( b1, a1, x256*POJACANJE[0])
    x2_eq = lfilter( b1, a1, x128*POJACANJE[1])
    x3_eq = lfilter( b1, a1, x64*POJACANJE[2])
    x4_eq = lfilter( b1, a1, x32*POJACANJE[3])
    x5_eq = lfilter( b1, a1, x16*POJACANJE[4])
    x6_eq = lfilter( b1, a1, x8*POJACANJE[5])
    x7_eq = lfilter( b1, a1, x4*POJACANJE[6])
    x8_eq = lfilter( b1, a1, x2*POJACANJE[7])
    x9_eq = lfilter( b1, a1, x1*POJACANJE[8])
    x10_eq = lfilter( b2, a2, x*POJACANJE[9])
    
    '''zato sto lfilter vraca razlicite duzine pri primenifiltra, 
    zato ako je neki niz duzi od N, uzece se samo N prvih odbiraka dok ce ostalih par da se odbaci
    dodato kao posledica sabiranja signala duzih od N
    '''
    
    x_total_eq = np.zeros(N)
    temp = dos_resample(x1_eq, False, 256)[:N]
    L = min(len(temp),N)
    x_total_eq[:L] +=  temp[:L]
    
    temp = dos_resample(x2_eq, False, 128)[:N]
    L = min(len(temp),N)
    x_total_eq[:L] +=  temp[:L]
    
    temp = dos_resample(x3_eq, False, 64)[:N]
    L = min(len(temp),N)
    x_total_eq[:L] +=  temp[:L]
    
    temp = dos_resample(x4_eq, False, 32)[:N]
    L = min(len(temp),N)
    x_total_eq[:L] +=  temp[:L]
    
    temp = dos_resample(x5_eq, False, 16)[:N]
    L = min(len(temp),N)
    x_total_eq[:L] +=  temp[:L]
    
    temp = dos_resample(x6_eq, False, 8)[:N]
    L = min(len(temp),N)
    x_total_eq[:L] +=  temp[:L]
    
    temp = dos_resample(x7_eq, False, 4)[:N]
    L = min(len(temp),N)
    x_total_eq[:L] +=  temp[:L]
    
    temp = dos_resample(x8_eq, False, 2)[:N]
    L = min(len(temp),N)
    x_total_eq[:L] +=  temp[:L]
    
    L = min(len(x9_eq), N)
    x_total_eq[:L] += x9_eq[:L]

    L = min(len(x10_eq), N)
    x_total_eq[:L] += x10_eq[:L]
        
    return x_total_eq


In [ ]:
from scipy.signal import lfilter
impuls = np.zeros(2**16)
impuls[0] = 1
h = IIR_equalizer2(impuls, fs, 'ROCK')
fig, ax = plt.subplots(figsize = (12,6))
ax.set_title('Impulsni odziv ekvalizatora')
ax.set_ylabel('h[n]')
ax.set_xlabel('n')
ax.plot(np.arange(len(h))[:2048],h[:2048]) 
plt.tight_layout()
plt.grid()
plt.show()

H = fft.fft(h)
H_db = 20 * np.log10(np.abs(H) + 1e-6)
f_full = np.arange(0,fs,fs/(len(h)))
fig, ax = plt.subplots(figsize = (12,6))
ax.set_title('Frekvencijska karakteristika ekvalizatora')
ax.set_ylabel('Amplituda [dB]')
ax.set_xlabel('f[Hz]')
ax.set_ylim(-7,7)
ax.plot(f_full[:len(h)//2], H_db[:len(h)//2])
ax.set_xscale('log')
plt.tight_layout()
plt.grid()
plt.show

In [ ]:
fs, rock = wavfile.read('dz2_signali/rock.wav')

rock_rock = IIR_equalizer2(rock, fs, 'ROCK')
print('Ovo je ROCK ekvalizacija:')
IPython.display.display(IPython.display.Audio(rock_rock, rate = fs))
rock_pop = IIR_equalizer2(rock, fs, 'POP')
print('Ovo je POP ekvalizacija:')
IPython.display.display(IPython.display.Audio(rock_pop, rate = fs))
rock_dance = IIR_equalizer2(rock, fs, 'DANCE')
print('Ovo je DANCE ekvalizacija:')
IPython.display.display(IPython.display.Audio(rock_dance, rate = fs))
rock_custom = IIR_equalizer2(rock, fs, 'CUSTOM')
print('Ovo je CUSTOM ekvalizacija:')
IPython.display.display(IPython.display.Audio(rock_custom, rate = fs))


In [ ]:
def dos_resample_rat(x, p ,q):
    #dizajn PM LP filtra
    fs = 44100
    '''za interpolaciju nam je potreban filtar sa Fa = 1/2p, 
    a za decimaciju filtar Fa=1/2a. Zato uzimamo gori slucaj od ova dva koji ce zadovoljiti
    oba zahteva.
    Fp je 0.8Fa zato sto su Fa i Fp normalizovane vrednostis'''
    Fa = min(1/(2*p), 1/(2*q))
    Fp = Fa*0.8
    Wa =  2 * Fa * np.pi
    Wp =  2 * Fp * np.pi
    Ap = 0.05
    Aa = 70
    
    dp = (10**(0.05*Ap)-1)/(10**(0.05*Ap)+1)
    da = 10**(-0.05*Aa)
    
    D = (0.005309*np.log10(dp)**2 + 0.07114*np.log10(dp) - 0.4761)*np.log10(da)
    D = D - (0.00266*np.log10(dp)**2 + 0.5941*np.log10(dp) + 0.4278)
    f = 11.01217 + 0.51244*(np.log10(dp) - np.log10(da))
    Bt = Wa-Wp
    M = int(np.ceil(2*np.pi*D/Bt - f*Bt/(2*np.pi) + 1))
    
    # Vektor relativnih ucestanosti za koje se zadaje Hid(w)
    F = [0, Fp, Fa, 0.5]
    # Zeljena idealna amplitudska karakteristika
    Hid = [1, 0]
    # Vektor tezinskih faktora
    Weight = [1, dp/da]
    
    Nfft = 1024
    
    filterFitsInSpecs = False
    #while koji pravi filtar, zatim ispituje da li su svi gabariti ispunjeni, ako nisu povecava red filtra za 1, i ponovo vrsi proveru
    while not filterFitsInSpecs:
        # Projektovanje filtra
        h = signal.remez(M, F, Hid, weight=Weight)
        W, H = signal.freqz(h, 1, worN = Nfft)
        Ha = abs(H)
        
        DeltaW = np.pi/Nfft
        kSB = int(np.ceil(Wa/DeltaW))
        kPB = int(np.floor(Wp/DeltaW))
        
        if  np.all(Ha[:kPB] > 1 - dp) and np.all(Ha[:kPB] < 1 + dp) and np.all(Ha[kSB:] < da):
            filterFitsInSpecs = True
        else:
            M += 1
    #resample naseg signala
    x_prosireno = np.zeros(len(x) * p)
    x_prosireno[::p] = x
    x_filtrirano = signal.lfilter(h, 1, x_prosireno) * p
    k = len(h) // 2
    x_bez_kasnjenja = x_filtrirano[k : -k]
    x_resampled = x_bez_kasnjenja[::q]
    
    return x_resampled

In [ ]:
#kreiranje test signala
f1 = 300
f2 = 1500
f3 = 7000
n = np.arange(4056)
test_signal = 2*np.cos(2*np.pi*f1/fs*n) + np.cos(2*np.pi*f2/fs*n) + 0.5*np.cos(2*np.pi*f2/fs*n)
# test za p>q 
signal_resampled1 = dos_resample_rat(test_signal, 5,3)
signal_resampled2 = dos_resample_rat(test_signal, 12, 5)
# test za p<q 
signal_resampled3 = dos_resample_rat(test_signal, 2,5)
signal_resampled4 = dos_resample_rat(test_signal, 3, 13)

In [ ]:
#plot dobijenih rezultata
fig, ax = plt.subplots(5,1, figsize=(12,10))
ax[0].set_title('Originalni signal')
ax[0].plot(np.arange(len(test_signal))/fs, test_signal)
ax[0].set_xlim(0,0.02)
ax[1].plot(np.arange(len(signal_resampled1))/fs, signal_resampled1)
ax[1].set_xlim(0,0.02)
ax[2].plot(np.arange(len(signal_resampled2))/fs, signal_resampled2)
ax[2].set_xlim(0,0.02)
ax[3].plot(np.arange(len(signal_resampled3))/fs, signal_resampled3)
ax[3].set_xlim(0,0.02)
ax[4].plot(np.arange(len(signal_resampled4))/fs, signal_resampled4)
ax[4].set_xlim(0,0.02)
ax[4].set_xlabel('t[s]')
ax[0].grid()
ax[1].grid()
ax[2].grid()
ax[3].grid()
ax[4].grid()
plt.tight_layout()
plt.show